# Laboratorio Integrador #2

En el siguiente laboratorio los estudiantes deben trabajar con un dataset de **baja numerosidad-baja dimensionalidad** y aplicar las técnicas de clusterización estudiadas en clases.

Objetivo:
*   Implementar un modelo de clasificación no supervisada sobre el espacio original de los datos.
*   Analizar los resultados de clasificación obtenidos por los algoritmos de clusterización (k-means, Hierarchical Clustering, GMM, Spectral Clustering).

Actividad Evaluativa:
1.   Es un trabajo individual.
2.   Implementar y aplicar los algoritmos k-means, Hierarchical Clustering, GMM, Spectral Clustering en el espacio original (***dataset(wq).xlsx***). 
3.   Medir el desempeño de clasificación para cada ecenario usando las tres métricas no supervisadas (Silhouette Coefficient, Calinski-Harabasz Index, Davies-Bouldin Index).
4.   Optimizar el número de clústers en el intervalo k=2..8 y plotear los resultados de clusterización (2D o 3D). Además, para el caso del k-means, mostrar el elbow plot, para el GMM, mostrar un plot con los valores del Bayesian Information Criterion (BIC) o el Akaike Information Criterion (AIC) para cada clusterización, y para el Hierarchical Clustering, mostrar el dendograma.
5.   Utilizar un algoritmo de reducción de dimensionalidad no lineal estudiado para reducir el espacio de clasificación a 3 componentes y aplicar el paso previo (punto 4) al nuevo espacio.
6.   Presentar un análisis de resultados de acuerdo a las métricas calculadas y plots generados donde se evidencie la comparativa entre ecenarios y la selección del número de clústers ideal.
7.   Subir el cuaderno desarrollado a la carpeta de trabajo en el D2L.


Valor:
*   25 puntos

Penalizaciones:
*   Cualquier error de implementación o concepto, se penalizará con -1 punto (acumulativo).
*   La no realización del punto 2 de la actividad se penalizará con el 45% (11.25 puntos).
*   La no realización del punto 3 de la actividad se penalizará con el 10% (2.5 puntos).
*   La no realización del punto 4 de la actividad se penalizará con el 15% (3.75 puntos).
*   La no realización del punto 5 de la actividad se penalizará con el 15% (3.75 puntos).
*   La no realización del punto 6 de la actividad se penalizará con el 15% (3.75 puntos).
*   La no realización del punto 7 de la actividad se penalizará con el 100% (25 puntos).



# *Setup*
Overall libraries for data handling, visualization, preparation, and evaluation

In [5]:
# Import libraries
'''Main'''
import numpy as np
import pandas as pd
import os, time
import io
import pickle, gzip

'''Data Viz'''
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl

%matplotlib inline

'''Data Prep and Model Evaluation'''
from sklearn import preprocessing as pp
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

# Load and Explore data

In [7]:
# Load datasets
file_path = 'wine-quality.csv'
data = pd.read_csv(file_path)
# data = pd.read_excel(file_path)
# Replace periods in large numbers and convert to numeric
data = data.applymap(lambda x: float(str(x).replace('.', '')) if isinstance(x, str) and x.replace('.', '').isdigit() else x)

# Check for any non-numeric columns and drop them or encode them if necessary
data = data.apply(pd.to_numeric, errors='coerce')
data = data.dropna()  # Drop rows with any NaN values (optional)

# Preview data
data.head()


/var/folders/w6/41d27j5d2lq8gg6n97xt284c0000gp/T/ipykernel_59504/1131745708.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: float(str(x).replace('.', '')) if isinstance(x, str) and x.replace('.', '').isdigit() else x)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [63]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


## Data preprocessing 

In [8]:
# Convert DataFrame to NumPy array for clustering and plotting
X = data.to_numpy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Clustering


---



## Original space

## k-Means

## Herarchical Clustering

## GMM

## Spectral Clustering

# Reduced space by a non-linear model

## k-Means

## Herarchical Clustering

## GMM

## Spectral Clustering

## Results and Discusion

# Conclusions